# Intro
#### This is a workbook about Credit Default Swaps (CDS). We will start by explaining what a CDS is. Next we will explain how they are used, and a history of how they have been used in the past. Finally, we will delve into the mathematics of pricing a CDS, and provide the python code to do so.

# Part 1: All about Credit Default Swaps

## What is a Credit Default Swap?
#### A CDS is a financial derivative that acts as insurance on a corporate bong.  The purchaser (often called the protection buyer) of the CDS pays premiums (referred to as the CDS Spread), usually on a quarterly basis. In return, if the reference company defaults or fails to make a bond payment, the seller (often reffered to as the protection seller) compensates the purchaser. 

## How are Credit Default Swaps used?
#### As stated above, the primary purpose of CDSs is as insurance against a company going bankrupt. In the same way one might hedge against the fall of a stock's price, or an increase in a commodity's price, the protection buyer is hedging against the default of the company. So far we have talked about how CDSs can be used to protect against the default of a company. However, a party can buy protection against many different bonds. For instance, in the same way a buyer can buy protection against the default of a company, someone can purchase protection against the bankruptcy of a government. Another famous example is that of Michael Burry. Burry bought CDSs referencing sub-prime Mortgage Backed Securities (MBS), correctly predicting the implosion of the housing market in 2008. 

## Layers of Risk
#### One interesting caveat of CDSs is that the event that they are supposed to protect against, bankruptcy, can also be in issue for the protection seller. What if the seller of the CDS goes bankrupt? 
#### This is what happened to American International Group (AIG) in 2008. AIG had sold so many CDSs that there was concern they would go bankrupt and not be able to pay the protection buyers. The question became, if AIG was not able to pay the purchasers of the CDSs, how many of those purchasers would also go bankrupt? Not wanting to find out the answer to that question, the US government bailed out AIG.

# Part 2: Pricing a Credit Default Swap

### Setup
#### The following is the necessary setup, we import the necessary libraries and read the data. I assume most of this is rather standard, I will focus a little bit on interp1d. Interp1d is used to interpolate data, our dummy data is broken up by year but payments are made quarterly. So, we interpolate the quarterly information. In reality, this is far from perfect, but for the purposes of this notebook it is fine. The maturities are kept in a seperate list so we can iterate over them later, as CDSs with different maturities have different values...duh.
#### The second part defines some constants. The recovery rate represents the amount an unprotected bond holder can expect to recoup if the reference entity defaults. 40% is fairly standard, but usually recovery rates change with the seniority of the bond. The notional value is the amount of money on which the protection is written. In the equation below it will be omitted for sake of normalizing the equations, but one simply multiplies the Notional value. Finally, dt represents how often we make payments. Usually they are quarterly payments, hence dt = .25.
#### Finally, the function defined at the bottom produces the survival probability, i.e. the probability the reference entity does not default before a given time. 

In [31]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
# Load the data
cds_data = pd.read_csv("cds_spreads.csv")
discount_data = pd.read_csv("discount_factors.csv")
hazard_data = pd.read_csv("dummy_hazard_rates.csv")
spread_interp = interp1d(cds_data["Maturity (Years)"], cds_data["CDS Spread (bps)"], kind='linear', fill_value="extrapolate")
discount_interp = interp1d(discount_data["Maturity (Years)"], discount_data["Discount Factor"], kind='linear', fill_value="extrapolate")
hazard_interp = interp1d(hazard_data["Maturity (Years)"], hazard_data["Hazard Rate"], kind='linear', fill_value="extrapolate")
maturities = cds_data["Maturity (Years)"].values




recovery_rate = 0.4
Notional = 1000000
dt = 0.25  # Quarterly payments



def get_Survival_Probability(hazard_rate, times):
    return np.exp(-hazard_rate*times)



## The Premium Leg
#### The premium leg of a CDS represents the payments the buyer makes to the seller. The mathematical representation of the premium leg is
$$
\sum_{i=1}^n D(t_i)Q(t_i)s\Delta_i~+~\sum_{i=1}^n D(t_i)(Q(t_{i-1}) - Q(t_i))\frac{s\Delta_i}{2}
$$
#### The first part of this equation represent the premium payments assuming the reference entity does not default. The second part of the equation takes into account what happen when the reference entity does default.

### Premium leg: Regular Premium
#### Let's examine the first part of the above equation
$$\sum_{i=1}^n D(t_i)Q(t_i)s\Delta_i
$$
- $D(t_i)$: The discount factor at time $t_i$
- $Q(t_i)$: The probability that the reference entity survives until time $t_i$
- $s \Delta_i$: The premium amount paid over the period $\Delta_i$

#### Summing over the entire period, we get the total present value of the regular premium

### Python for the Regular Premium leg
#### The for loop just iterates over all the different maturities. The next 4 lines are boiler plate setup. Let's focus on the 6th line.
- First we sum over all the different time intervals using np.sum()
- Notional* (spread/10000)*dt is the premium amount paid that quarter. We divide by 10000 to ensure we are in the proper units. 
- discount_factors is the array of discount factors
- Finally, we use the get_Survival_Probability function to get the list of survival probabilities

In [40]:
prem_reg = np.array([])
for T in maturities:
    times = np.arange(dt, T + dt, dt)
    spread = spread_interp(T)
    discount_factors = discount_interp(times)
    hazard_rate = hazard_interp(T)
    prem_surv = np.sum(Notional*(spread / 10000) * dt * discount_factors * get_Survival_Probability(hazard_rate, times))
    prem_reg = np.append(prem_reg,prem_surv)
print(prem_reg)

[  7795.17656453  18094.55931405  30656.5529161   45320.45835515
  61854.67123419  97103.41527881 148450.65710306]


### Premium leg: Accrued Premium
#### The second part of the above equation adjusts for the partial payments of times when an entity does defualt, i.e. if an entity defaults mid quarter, some payment is still made by the protection purchaser. Let's examine the equation below.
$$\sum_{i=1}^n D(t_i)(Q(t_{i-1}) - Q(t_i))\frac{s\Delta_i}{2}
$$
- $D(t_i)$: The discount factor at time $t_i$
- $Q(t_{i-1}) - Q(t_i)$: The probability of a default occuring between $(t_{i-1}, t_i)$
- $\frac{s \Delta_i}{2}$: The premium amount paid over the period $\frac{\Delta_i}{2}$. We divide by 2 as we assume the reference entity defaults halfway through the quarter.

#### Summing over the entire period, we get the total present value of the accrued premium

### Python for the Accrued Premium leg
#### The first 5 lines are the same, again, let's focus on the 6th line.
- First we sum over all the different time intervals using np.sum()
- $Notional* (spread/10000)*dt*.5$ is the premium amount paid assuming the entity defaults halfway through the quarter. We divide by 10000 to ensure we are in the proper units. 
- discount_factors[1:]: The discount factors, notice how we start at index 1.
- The last part represent the probabilty of a default occuring during a time interval. The first part is all the survival probabilities except the last, because no time interval starts at time n. The second part is all of the survival probabilityes except the first, because no time intval ends at time 0.

In [41]:
prem_accrued = np.array([])
for T in maturities:
    times = np.arange(dt, T + dt, dt)
    spread = spread_interp(T)
    discount_factors = discount_interp(times)
    hazard_rate = hazard_interp(T)
    prem_def = np.sum(Notional * discount_factors[1:] * (spread/10000)*dt*.5*(get_Survival_Probability(hazard_rate,times[:-1]) - get_Survival_Probability(hazard_rate, times[1:])))
    prem_accrued = np.append(prem_accrued,prem_def)
print(prem_accrued)

[  8.73728333  27.60309044  55.98804867  89.93407979 131.6882418
 221.46886161 360.29948596]


## The Protection Leg
#### The protection leg of a CDS is the payment the protection seller makes to the protection buyer in the case of a default. The mathematical representation of the protection leg is
$$
(1-R)\sum_{i=1}^n D(t_i)(Q(t_{i-1}) - Q(t_i))
$$
#### Lets go through this piece by piece in order to understand it. 
- $(1-R)$: Here, R is the recovery rate i.e. the amount the bond holder is expected to get back when a default occurs. So (1-R), would be the amount the bond holder is expected to lose. The seller of a CDS is only responsible to pay back (1-R)\% of the bond holders losses. Usually, R is determined by the seniority of the bond.

#### To explain the other pieces, we will examine them in the time interval $(t_{i-1}, t_i)$
- $Q(t_{i-1}) - Q(t_i)$: The probability of a default occuring between $(t_{i-1}, t_i)$
- $D(t_i)$: The discount factor at time $t_i$, if a default does occur we must discount the payment

#### By taking the sum over all of the intervals, we get the present value of the protection leg.

### Python for the Protection leg
#### The first 5 lines are the same set up we saw for parts of the premium leg. Let's focus on the 6th line. 
- Similarly to the code above, we sum over the different time intervals using np.sum(). 
- (1 - recovery_rate) obviously represents (1-R). Here we choose R = .4, which is fairly standard
- The notional represents the same as in the premium leg
- discount_factors[1:]: The discount factors, notice how we start at index 1.
- Again, the last part represent the probabilty of a default occuring during a time interval. The first part is all the survival probabilities except the last, because no time interval starts at time n. The second part is all of the survival probabilityes except the first, because no time intval ends at time 0. 

In [42]:
protection= np.array([])
for T in maturities:
    times = np.arange(dt, T + dt, dt)
    spread = spread_interp(T)
    discount_factors = discount_interp(times)
    hazard_rate = hazard_interp(T)
    prot_leg = np.sum((1 - recovery_rate)*Notional * discount_factors[1:] *
                      (get_Survival_Probability(hazard_rate,times[:-1]) - get_Survival_Probability(hazard_rate, times[1:])))
    protection = np.append(protection,prot_leg)
print(protection)

[ 5242.37000008 13946.82464413 24431.14850923 34534.68663916
 45150.25433078 64427.30519637 91023.02803135]


### Net Present Value
#### The NPV of a CDS is the premium leg - protection. It is the current value of the contract. When a CDS is created the NPV is 0, however, as conditions change, say is a reference entity has a change in credit rating, the NPV changes.

In [44]:
NPV = prem_reg + prem_accrued - protection
NPV

array([ 2561.54384778,  4175.33776037,  6281.39245553, 10875.70579578,
       16836.10514521, 32897.57894405, 57787.92855766])